In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import csv
import plotly.express as px
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as mse 
from  skopt import BayesSearchCV


In [2]:
development = pd.read_csv(r'C:\Users\feder\Desktop\POLI\DSL\DSL_Winter_Project_2024\DSL_Winter_Project_2024\development.csv')
evaluation = pd.read_csv(r'C:\Users\feder\Desktop\POLI\DSL\DSL_Winter_Project_2024\DSL_Winter_Project_2024\evaluation.csv')

In [3]:
df = pd.concat([development, evaluation], sort=False)

x              False
y              False
pmax[0]        False
negpmax[0]     False
area[0]        False
               ...  
pmax[17]       False
negpmax[17]    False
area[17]       False
tmax[17]       False
rms[17]        False
Length: 92, dtype: bool

In [4]:
df.set_index('Id', inplace=True)

Id             False
pmax[0]        False
negpmax[0]     False
area[0]        False
tmax[0]        False
               ...  
pmax[17]       False
negpmax[17]    False
area[17]       False
tmax[17]       False
rms[17]        False
Length: 91, dtype: bool

In [6]:
outliers_idx = [0,7,12,15,16,17]
dropped_cols = ['pmax[%s]' %i for i in outliers_idx]+['negpmax[%s]' %i for i in outliers_idx]+['area[%s]' %i for i in outliers_idx]+['tmax[%s]' %i for i in outliers_idx]+['rms[%s]' %i for i in outliers_idx]
print(dropped_cols)

['pmax[0]', 'pmax[7]', 'pmax[12]', 'pmax[15]', 'pmax[16]', 'pmax[17]', 'negpmax[0]', 'negpmax[7]', 'negpmax[12]', 'negpmax[15]', 'negpmax[16]', 'negpmax[17]', 'area[0]', 'area[7]', 'area[12]', 'area[15]', 'area[16]', 'area[17]', 'tmax[0]', 'tmax[7]', 'tmax[12]', 'tmax[15]', 'tmax[16]', 'tmax[17]', 'rms[0]', 'rms[7]', 'rms[12]', 'rms[15]', 'rms[16]', 'rms[17]']


In [7]:
df = df.drop(columns=dropped_cols)
display(df)

,x,y,pmax[1],negpmax[1],area[1],tmax[1],rms[1],pmax[2],negpmax[2],area[2],...,pmax[13],negpmax[13],area[13],tmax[13],rms[13],pmax[14],negpmax[14],area[14],tmax[14],rms[14]
Id,,,,,,,,,,,,,,,,,,,,,
NaN,200.0,200.0,5.409161,-17.721210,4.538778,22.600000,1.563902,6.084506,-19.892543,3.132328,...,7.454877,-16.861163,4.191909,72.600000,1.184943,6.611877,-17.685799,4.884680,162.800000,1.284969
NaN,200.0,200.0,4.414289,-4.736827,3.720435,111.875058,1.228058,4.507257,-3.787175,3.267461,...,18.472514,-3.372324,16.620582,119.812512,1.578009,13.802252,-5.154840,8.758870,120.000000,1.093953
NaN,200.0,200.0,4.567191,-5.985437,3.490490,107.800000,1.015408,5.384155,-3.948853,2.929150,...,7.336668,-5.742383,11.792258,71.908212,1.333036,5.940039,-3.860550,5.340140,6.614830,1.183951
NaN,200.0,200.0,5.019058,-4.229949,6.786200,175.600000,1.350620,4.165598,-5.576041,1.544370,...,8.450671,-6.342114,4.496741,72.600000,1.945065,6.434910,-5.576315,15.789638,72.195420,1.738998
NaN,200.0,200.0,3.250262,-5.783587,2.449456,159.800000,0.759515,4.736023,-5.137939,4.211047,...,8.721952,-4.396436,9.127136,71.411535,1.513988,4.483080,-5.361823,3.366542,160.185705,1.323621
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128495.0,NaN,NaN,9.327438,-3.704605,6.657654,71.400000,0.891158,4.763165,-4.856036,11.514778,...,54.351236,-27.172751,26.922277,71.400000,1.085946,22.433310,-9.949075,13.170802,71.200000,1.747292
128496.0,NaN,NaN,5.833882,-6.185739,4.184656,20.037256,1.422575,3.790421,-5.154848,4.475711,...,15.090143,-6.553778,8.782450,71.600000,1.425143,4.467065,-4.729529,2.264683,84.400000,1.617628
128497.0,NaN,NaN,19.402353,-8.160696,11.304202,70.800000,1.370319,17.555460,-8.818259,9.256798,...,23.518393,-9.069559,13.958058,70.800000,1.625532,4.901157,-4.210786,2.049936,66.800000,0.908640


In [8]:
coord=['x', 'y']
test_mask = df['x'].isna() & df['y'].isna()

X = df.drop(columns=coord).values
y = df[coord].values

X_train_valid = X[~test_mask]
y_train_valid = y[~test_mask]

X_test = X[test_mask]
y_test = y[test_mask]

from sklearn.model_selection import train_test_split
rs = 2024

X_train, X_valid, y_train, y_valid = train_test_split(X_train_valid,y_train_valid,shuffle=True, random_state=rs)

In [ ]:
parameters = {"n_estimators":[50,100], "max_depth":[50,100], "min_samples_split": [2,5,10], "max_features":[ "sqrt", "log2"], "random_state":[rs], "n_jobs":[-1] } 
bayes = BayesSearchCV(RandomForestRegressor(), search_spaces=parameters, scoring="r2", cv=5, n_iter=20, n_jobs=-1, random_state=rs)

In [ ]:
bayes.best_params_

insert the values you see above

In [ ]:
rfr = RandomForestRegressor(n_estimators=,min_samples_split= ,max_features='',max_depth= ,random_state=rs)
rfr.fit(X_train, y_train)
y_rfr=rfr.predict(X_test)
distance=np.mean(np.sqrt(np.sum((y_rfr - y_test) ** 2, axis=1)))
print(distance)

eva_y=rfr.predict(eva_df)

df = pd.DataFrame(eva_y, columns=['Id', 'Predicted'])
df['Predicted'] = df.apply(lambda row: f"{row['Id']:.1f}|{row['Predicted']:.1f}", axis=1)
df['Id'] = df.index
df.to_csv('outputbayes0.csv', index=False, sep=',')
